**Repo Directory**

In [ ]:
# !git clone https://github.com/abualmun/ksa-sights-classification

In [2]:
directory = "."


**Import Libraries**

In [3]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models 
import torch.nn as nn
import numpy as np

**Check using cuda GPU**

In [4]:

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


**Load data and tarnsforms**

In [5]:

mean = ([0.4863, 0.4894, 0.4880])
std =  ([0.2132, 0.2025, 0.2220])

transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])

dataset = datasets.ImageFolder(root=directory+'/locations', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



**Mean and STD Calculation**

In [ ]:

# mean = 0.0
# std = 0.0
# total_images_count = 0
# for images, _ in train_loader:
#     images = images.view(images.size(0), images.size(1), -1)
#     mean += images.mean(2).sum(0)
#     std += images.std(2).sum(0)
#     total_images_count += images.size(0)
# mean /= total_images_count
# std /= total_images_count
# print(mean)
# print(std)

# mean = ([0.4863, 0.4894, 0.4880])
# std =  ([0.2132, 0.2025, 0.2220])


tensor([0.4863, 0.4894, 0.4880])
tensor([0.2132, 0.2025, 0.2220])


**Load Model**

In [6]:
# Load the model and move it to the GPU
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=14)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


/home/abualmun/env/main/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/abualmun/env/main/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


**Start Training**

In [7]:

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        # Move images and labels to GPU
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/10], Loss: 1.3468
Epoch [2/10], Loss: 0.8396
Epoch [3/10], Loss: 0.5720
Epoch [4/10], Loss: 0.4306
Epoch [5/10], Loss: 0.3430
Epoch [6/10], Loss: 0.2590
Epoch [7/10], Loss: 0.2098
Epoch [8/10], Loss: 0.1792
Epoch [9/10], Loss: 0.0988
Epoch [10/10], Loss: 0.1358


**Validation**

In [8]:

# Validation (optional)
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        # Move images and labels to GPU
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Validation Accuracy: {100 * correct / total:.2f}%')

Validation Accuracy: 73.97%


**Save the model's state dictionary**

In [9]:
torch.save(model.state_dict(), directory+'14_classes_model.pth')